In [ ]:
--=============== МОДУЛЬ Итоговое задание. POSTGRESQL =======================================
--1.Выведите названия самолетов, которые имеют менее 50 посадочных мест
select a.model as "Название самолета", count(seat_no) as "Количество мест"
from aircrafts a
join seats s using(aircraft_code)
group by a.model
having count(seat_no) < 50
order by a.model

--2.Выведите процентное изменение ежемесячной суммы бронирования билетов
with c1 as
(select date_trunc('month',book_date) as "date",
sum(total_amount) as "sum",
sum(total_amount) - lag(sum(total_amount),1,0.) over(order by date_trunc('month',book_date)) as "diff",
row_number () over ()
from  bookings
group by date_trunc('month',book_date))
select c1.date, c1.sum,
	case
		when c1.row_number = 1 then 100
		when c1.row_number > 1 then round(c1.diff*100/lag(c1.diff) over(),2)
	end as "percent"
from  c1

--3.Выведите названия самолетов, не имеющих бизнесс-класс
select s.fare_conditions, array_agg(model) models
from aircrafts a
join seats s using(aircraft_code)
where fare_conditions != 'Business'
group by s.fare_conditions

--4. Вывести накопительный итог количества мест в самолетах по каждому аэропорту на каждый день,
--учитывая только те самолеты, которые летали пустыми и только те дни, где из одного аэропорта таких самолетов вылетало более одного--


with c2 as---5.выводим накопительный итог
(select distinct*---4.выводим дни, где из одного аэропорта пустых самолетов вылетало более одного
from
(with c1 as
(select * ---3.находим пустые самолеты
from
(select departure_airport as "d_airport", f.aircraft_code as "a_code", actual_departure as "a_departure", count(s.seat_no) as kol_vo_mest
from flights f
join seats s on f.aircraft_code = s.aircraft_code
where actual_departure is not null
group by 1,2,3
order by actual_departure)t101----1.находим все самолеты
full outer join
(select array_agg(t.passenger_id), f.departure_airport, f.actual_departure, a.aircraft_code
from tickets t
join boarding_passes bp on t.ticket_no = bp.ticket_no
join flights f  on bp.flight_id = f.flight_id
join aircrafts a on f.aircraft_code = a.aircraft_code
group by 2,3,4
order by actual_departure)t102 ---2.выводим только те самолеты, которые были с людьми
on t101.a_code = t102.aircraft_code and t101.a_departure = t102.actual_departure and t101.d_airport = t102.departure_airport
where array_agg is null)
select c1.d_airport, c1.a_code, c1.a_departure::date, c1. kol_vo_mest, sum(count(a_code)) over(partition by d_airport, a_departure::date) as "kol_vo_sam"
from c1
group by c1.d_airport, c1.a_code, c1.a_departure, c1.kol_vo_mest)t103
where kol_vo_sam > 1)
select c2.d_airport, c2.a_departure::date, c2.kol_vo_mest,
sum(c2.kol_vo_mest) over(partition by c2.d_airport order by c2.a_departure::date) as "Накопительный итог"
from c2
order by 1,2



--5.Найдите процентное соотношение перелетов по маршрутам от общего количества перелетов

select round(t1.flight_id_count*100/t1.sum,2) as "percent", t1.route
from
(with c1 as
(select flight_id, concat(departure_airport, '-', arrival_airport) as "route"
from flights f
group by route, flight_id),
c2 as (select sum(count(c1.flight_id)) over(partition by c1.route) as "flight_id_count", c1.route
from c1
group by c1.route)
select c2.flight_id_count, c2.route, sum(c2.flight_id_count) over()
from c2)t1


--6.Выведите количество пассажиров по каждому коду сотового оператора
with cte as
(select contact_data::json ->> 'phone' as "phones", passenger_id
from tickets)
select count(cte.passenger_id) as "Количество людей",  substring(phones from 3 for 3) as "code"
from cte
group by code
order by code

--7.Классифицируйте финансовые обороты (сумма стоимости билетов) по маршрутам:
--До 50 млн - low
--От 50 млн включительно до 150 млн - middle
--От 150 млн включительно - high

select count(route) as "Количество маршрутов", f_obor
from
(with c1 as
(select distinct concat(f.departure_airport, ' - ', f.arrival_airport) as "route",
sum(amount) as "amount"
from flights f
join ticket_flights tf using(flight_id)
group by 1)
select c1.*,
case
		when sum(c1.amount) over(partition by c1.route) < 50000000  then 'low'
		when sum(c1.amount) over(partition by c1.route) between 50000000 and 150000000 then 'middle'
		when sum(c1.amount) over(partition by c1.route) >= 150000000 then 'high'
	end as "f_obor"
	from c1)t1
	group by f_obor


--8.Вычислите медиану стоимости билетов, медиану размера бронирования и отношения медианы бронировaния к медиане стоимости билетов, округленной до сотых

with cte as
(select percentile_cont(0.5) within group(order by amount) as "ticket_med",
percentile_cont(0.5) within group(order by total_amount) as "booking_med"
from ticket_flights tf
join tickets t using(ticket_no)
join bookings b using(book_ref))
select cte.ticket_med, cte.booking_med, round(cte.booking_med::numeric/cte.ticket_med::numeric, 2) as "percent"
from cte



--9.Найдите значение минимальной стоимости полета 1 км для пассажиров. То есть нужно найти расстояние между аэропортами и с учетом стоимости билетов получить искомый результат


with cte as
(select f.departure_airport, f.arrival_airport, f.aircraft_code,
	earth_distance(ll_to_earth(d.latitude,d.longitude), ll_to_earth(a.latitude,a.longitude)),
	f.flight_no, f.flight_id,
	d.airport_name as departure_airport_name,
	a.airport_name as arrival_airport_name
	from
	flights f,
	airports d,
	airports a
	where f.departure_airport = d.airport_code and f.arrival_airport = a.airport_code)
	select distinct cte.departure_airport_name, cte.arrival_airport_name, cte.dist_airport,
	tf.amount, round(tf.amount/cte.dist_airport,2) as "price_one_km", min(round(tf.amount/cte.dist_airport,2)) over()
	from ticket_flights tf
	join cte on cte.flight_id = tf.flight_id




